# Download Historical OHLC Data

In [1]:
import pandas as pd
from breeze_connect import BreezeConnect
import mysql.connector as sqlConnector
import datetime
from datetime import date
import time
from selenium import webdriver
from pyotp import TOTP
import urllib
import warnings
warnings.filterwarnings('ignore')

Login

In [2]:
file = open("security.txt", "r")
keys = file.read().split()  # Get a List of keys
api_key = keys[0]
key_secret = keys[1]
userID = keys[2]
pwd = keys[3]
totp_key = keys[4]
breeze = BreezeConnect(api_key=api_key)

In [3]:
browser = webdriver.Chrome()
browser.get("https://api.icicidirect.com/apiuser/login?api_key="+urllib.parse.quote_plus(api_key))
browser.implicitly_wait(5)
username = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[1]/input')
password = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[3]/div/input') 
username.send_keys(userID)
password.send_keys(pwd)
#Checkbox
browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[4]/div/input').click()
# Click Login Button
browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[5]/input[1]').click()
time.sleep(2)
pin = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[2]/div/div[2]/div[2]/div[3]/div/div[1]/input')
totp = TOTP(totp_key)
token = totp.now()
pin.send_keys(token)
browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[2]/div/div[2]/div[2]/div[4]/input[1]').click()
time.sleep(3)
temp_token=browser.current_url.split('apisession=')[1][:8]
print('temp_token', temp_token)

In [4]:
breeze.generate_session(api_secret=key_secret,session_token=temp_token)

In [6]:
from sqlalchemy import create_engine
username = '******'
password = '*****'
database_name = "algo"
hostname = "127.0.0.1"
engine = create_engine(f"mysql+pymysql://{username}:{password}@{hostname}:3306/{database_name}")

intervals: "1second","1minute", "5minute", "30minute" or as "1day".<br>

Maximum candle intervals in one single request is 1000

### Download data refined function

In [7]:
adate = '2023-03-17'
symbol = 'TCS'
data = breeze.get_historical_data(interval="5minute", from_date= adate + "T09:15:00.000Z",
                            to_date= adate + "T15:30:00.000Z", stock_code=symbol, exchange_code="NSE", product_type="cash")
put_data = pd.DataFrame(data["Success"])
if put_data.empty:
    print('Data not found:', adate)
    df = pd.DataFrame()
else:
    df = put_data[['datetime', 'open', 'high', 'low', 'close']]


In [8]:
df.head()

,datetime,open,high,low,close
0,2023-03-17 09:10:00,3150.5,3150.5,3150.5,3150.5
1,2023-03-17 09:15:00,3159.85,3162.5,3151.35,3158.35
2,2023-03-17 09:20:00,3159.05,3186.7,3158.95,3165.85
3,2023-03-17 09:25:00,3165.95,3194,3165.35,3182
4,2023-03-17 09:30:00,3182.05,3189,3172.75,3179.95


In [9]:
df.to_sql('daily_ohlc', con=engine, if_exists='append', index=False, chunksize=1000)
print('Saved to Database')

Saved to Database


In [10]:
def download_NFO(symbol, sdate, edate, expiry_date):
    #data = breeze.get_historical_data_v2(interval="1minute",
    data = breeze.get_historical_data_v2(interval="1minute",
                                from_date= sdate,
                                to_date= edate,
                                stock_code= symbol,
                                exchange_code="NFO",
                                product_type="futures",
                                expiry_date=expiry_date)
    #print(data)
    put_data = pd.DataFrame(data["Success"])
    if put_data.empty:
        print('Data not found:', adate)
        df = pd.DataFrame()
    else:
        df = put_data[['datetime', 'open', 'high', 'low', 'close']]
        df.columns= ['DateTime', 'Open', 'High', 'Low', 'Close']
        df['DateTime'] = pd.to_datetime(df['DateTime'])
        df.set_index('DateTime', inplace=True)
        df['volume'] = 0
        return df

In [11]:
edate = datetime.datetime(2023, 3, 17, 11, 0, 0)
sdate = datetime.datetime(2023, 3, 17, 9, 15, 0)
edate_iso = edate.isoformat()[:19] + '.000Z'
sdate_iso = sdate.isoformat()[:19] + '.000Z'
expiry_date = datetime.date(2023, 3, 29)
xdate_iso = expiry_date.isoformat()[:10] + 'T05:30:00.000Z'
sdate_iso, edate_iso, xdate_iso

('2023-03-17T09:15:00.000Z',
 '2023-03-17T11:00:00.000Z',
 '2023-03-29T05:30:00.000Z')

In [12]:
df_fut = download_NFO("TCS", sdate_iso, edate_iso, xdate_iso)
df_fut.head()

,Open,High,Low,Close,volume
DateTime,,,,,
2023-03-17 09:15:00,3167.65,3190.20,3155.00,3190.20,0
2023-03-17 09:16:00,3192.25,3199.50,3185.15,3192.85,0
2023-03-17 09:17:00,3191.90,3201.80,3191.05,3192.05,0
2023-03-17 09:18:00,3193.10,3193.45,3181.00,3181.75,0
2023-03-17 09:19:00,3181.00,3188.50,3181.00,3187.40,0


In [13]:
df_fut.tail()

,Open,High,Low,Close,volume
DateTime,,,,,
2023-03-17 10:56:00,3187.10,3188.20,3187.10,3187.75,0
2023-03-17 10:57:00,3188.85,3190.00,3188.30,3189.85,0
2023-03-17 10:58:00,3189.85,3193.15,3189.45,3192.10,0
2023-03-17 10:59:00,3192.10,3192.90,3191.75,3192.05,0
2023-03-17 11:00:00,3192.05,3193.25,3190.65,3191.55,0


In [14]:
df_fut.to_csv('fut.csv')

In [15]:
def gethistoricaldata_options(interval, from_date, to_date, stock_code, right, expiry_date, strike_price):
    data = breeze.get_historical_data(interval= interval,
                                from_date= from_date,
                                to_date= to_date,
                                stock_code= stock_code,
                                exchange_code="NFO",
                                product_type="options",
                                expiry_date=expiry_date,
                                right=right,
                                strike_price=strike_price)
    put_data = pd.DataFrame(data["Success"])
    if put_data.empty:
        print('Data not found:', to_date)
        df = pd.DataFrame()
        return df
    else:
        df = put_data[['datetime', 'open', 'high', 'low', 'close']]
    return df 


In [16]:
tdate = datetime.datetime.now()
fdate = tdate - datetime.timedelta(days=2)
expiry_date = datetime.date(2023, 3, 29)
tdate_iso = tdate.isoformat()[:10] + 'T05:30:00.000Z'
fdate_iso = fdate.isoformat()[:10] + 'T05:30:00.000Z'
expiry_date_iso = expiry_date.isoformat()[:10] + 'T05:30:00.000Z'
icici_code = breeze.get_names(exchange_code = 'NSE',stock_code = 'INFY').get('isec_stock_code')
gethistoricaldata_options('5minute',fdate_iso, tdate_iso, icici_code, "call", expiry_date_iso, 1500)

Data not found: 2024-08-12T05:30:00.000Z


""
